In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json

In [3]:
dirr = 'osti.gov_13670'
result_log = json.load(open(f'writes/{dirr}/result_log.json', 'r'))
log = result_log['log'][1]
updated_code = log['updated']
with open(f'test/overrided.js', 'w+') as f:
    f.write(updated_code)